# Lección 1: Programando un agente ReAct (Reason+Act) desde cero

In [ ]:
# basado en https://til.simonwillison.net/llms/python-react-pattern

In [ ]:
!pip install python-dotenv openai httpx

In [ ]:
import openai
import re
import httpx
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

from openai import OpenAI

In [ ]:
client = OpenAI()

In [ ]:
chat_completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Who are you?"}]
)

In [ ]:
chat_completion.choices[0].message.content

'I am an AI language model created by OpenAI, designed to assist with a variety of inquiries by providing information, answering questions, and engaging in conversation. How can I help you today?'

In [ ]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
                        model="gpt-4o",
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content


In [ ]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 10 kg

You then output:

Answer: A bulldog weights 10 kg
""".strip()

In [ ]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier":
        return("Scottish Terriers average 9 kg")
    elif name in "Border Collie":
        return("a Border Collies average weight is 16 kg")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 3 kg")
    else:
        return("An average dog weights 20 kg")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [ ]:
abot = Agent(prompt)

In [ ]:
result = abot("How much does a toy poodle weigh?")
print(result)

Thought: I should look up the average weight of a Toy Poodle using the average_dog_weight action.
Action: average_dog_weight: Toy Poodle
PAUSE


In [ ]:
result = average_dog_weight("Toy Poodle")

In [ ]:
result

'a toy poodles average weight is 3 kg'

In [ ]:
next_prompt = "Observation: {}".format(result)

'Observation: a toy poodles average weight is 3 kg'

In [ ]:
abot(next_prompt)

'Answer: A Toy Poodle weighs on average 3 kg.'

In [ ]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 10 kg\n\nYou then output:\n\nAnswer: A bulldog weights 10 kg'},
 {'role': 'user', 'content': 'How much does a to

In [ ]:
abot = Agent(prompt)

In [ ]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

'Thought: I need to find the average weight of both a Border Collie and a Scottish Terrier, then add them together to find the combined weight.\nAction: average_dog_weight: Border Collie\nPAUSE'

In [ ]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: a Border Collies average weight is 16 kg


In [ ]:
abot(next_prompt)

'Action: average_dog_weight: Scottish Terrier\nPAUSE'

In [ ]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

Observation: Scottish Terriers average 9 kg


In [ ]:
abot(next_prompt)

'Thought: Now that I have the average weights of both the Border Collie and the Scottish Terrier, I can calculate their combined weight.\nAction: calculate: 16 + 9\nPAUSE'

In [ ]:
next_prompt = "Observation: {}".format(calculate("16 + 9"))
print(next_prompt)

Observation: 25


In [ ]:
abot(next_prompt)

'Answer: The combined average weight of a Border Collie and a Scottish Terrier is 25 kg.'

### Add loop

In [ ]:
action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to selection action

In [ ]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a)
            for a in result.split('\n')
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [ ]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

Thought: I need to find the average weight of both a Border Collie and a Scottish Terrier, then add them together to find their combined weight.
Action: average_dog_weight: Border Collie
PAUSE
 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 16 kg
Action: average_dog_weight: Scottish Terrier
PAUSE
 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 9 kg
Action: calculate: 16 + 9
PAUSE
 -- running calculate 16 + 9
Observation: 25
Answer: The combined average weight of a Border Collie and a Scottish Terrier is 25 kg.
